# Custom Types

## Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(**Note that types can not be redefined** - you have to restart your Julia session to change a type definiton.)

In [ ]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/v1/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [ ]:
methods(MyType)

In [ ]:
m = MyType()

In [ ]:
typeof(m)

In [ ]:
m isa MyType

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [ ]:
struct Point
    x::Float64
    y::Float64
end

In [ ]:
Point()

The default constructor always expects values for all fields.

In [ ]:
Point(1.2, 3.4)

In [ ]:
p = Point(1.2, 3.4)

In [ ]:
# a.<TAB>
p.x

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [ ]:
p.x = 2

In [ ]:
mutable struct Point2
    x::Float64
    y::Float64
end

In [ ]:
p = Point2(1.2, 3.4)

In [ ]:
p.x = 4

In [ ]:
p

Abstract types are just as easy to define using the keyword `abstract type`.

In [ ]:
abstract type AbstractPoint end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [ ]:
struct Point3 <: AbstractPoint
    x::Float64
    y::Float64
end

In [ ]:
c = Point3(1.2, 3.4)

In [ ]:
c isa AbstractPoint

In [ ]:
supertype(Point3)

In [ ]:
subtypes(AbstractPoint)

## Example: Diagonal Matrix

In [ ]:
struct DiagMat
    diag::Vector{Float64}
end

In [ ]:
DiagMat([1.2,4.3,5.0])

### Arithmetic

In [ ]:
import Base: +, -, *, /

+(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag .+ Db.diag)
-(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag .- Db.diag)
*(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag .* Db.diag)
/(Da::DiagMat, Db::DiagMat) = DiagMat(Da.diag ./ Db.diag)

In [ ]:
D1 = DiagMat([1,2,3])
D2 = DiagMat([2.4,1.9,5.7])

In [ ]:
D1 + D2

In [ ]:
D1 - D2

In [ ]:
D1 * D2

In [ ]:
D1 / D2

Arithmetics involving other types:

In [ ]:
# Number
*(x::Number, D::DiagMat) = DiagMat(x * D.diag)
*(D::DiagMat, x::Number) = DiagMat(D.diag * x)
/(D::DiagMat, x::Number) = DiagMat(D.diag / x)

# Vector
*(D::DiagMat, V::AbstractVector) = D.diag .* V

In [ ]:
D1 * 2

In [ ]:
D1 * rand(3)

Note that some functions already work for our `DiagonalMat`:

In [ ]:
sum([D1, D2])

### Parameterization

In [ ]:
DiagMat([1,2,3]) # implicit conversion to Vector{Float64}

In [ ]:
DiagMat([1+3im, 4-2im, im])

In [ ]:
DiagMat(["Why", "not", "support", "strings?"])

We can easily relax our type definition to allow all sorts of internal value types.

In [ ]:
struct DiagMatParam{T, V<:AbstractVector{T}}
    diag::V
end

Let's (again) define some arithmetics.

In [ ]:
# Essentially copied from above
import Base: +, -, *, /
+(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag .+ Db.diag)
-(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag .- Db.diag)
*(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag .* Db.diag)
/(Da::DiagMatParam, Db::DiagMatParam) = DiagMatParam(Da.diag ./ Db.diag)
# Number
*(x::Number, D::DiagMatParam) = DiagMatParam(x * D.diag)
*(D::DiagMatParam, x::Number) = DiagMatParam(D.diag * x)
/(D::DiagMatParam, x::Number) = DiagMatParam(D.diag / x)
# Vector
*(D::DiagMatParam, V::AbstractVector) = D.diag .* V

In [ ]:
DiagMatParam([1+3im, 4-2im, im])

In [ ]:
DiagMatParam(["This ", "just "]) * DiagMatParam(["should", "work!"])

### Duck typing: `AbstractArray`

Let's integrate our diagonal matrix into Julia's type hierarchy by subtyping `AbstractArray{T,2}` aka `AbstractMatrix{T}`.

Of course, our diagonal matrix type should then better behave as an array! A minimal list of methods that we should define are specified by the [`AbstractArray` interface](https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1)!

In [ ]:
struct DiagonalMatrix{T, V<:AbstractVector{T}} <: AbstractMatrix{T}
    diag::V
end

In [ ]:
# implement AbstractArray interface
Base.size(D::DiagonalMatrix) = (length(D.diag), length(D.diag))

function Base.getindex(D::DiagonalMatrix{T,V}, i::Int, j::Int) where {T,V}
    if i == j
        r = D.diag[i]
    else
        r = zero(T)
    end
    return r
end

function setindex!(D::DiagonalMatrix, v, i::Int, j::Int)
    if i == j
        D.diag[i] = v
    else
        throw(ArgumentError("cannot set off-diagonal entry ($i, $j)"))
    end
    return v
end

In [ ]:
D = DiagonalMatrix([1,2,3])

Note how it's automagically pretty printed!

In [ ]:
D * D

In [ ]:
D + D

In [ ]:
D - D

In [ ]:
D / D

Basic arithmetics **just works!** What about broadcasting and more complicated functions?

In [ ]:
sin.(D)

In [ ]:
sum([D, D, D])

In [ ]:
using LinearAlgebra
eigen(D)

It is still advantageous to define fast versions that utilize the special diagonal structure:

In [ ]:
@which D + D

In [ ]:
@which 3 * D

In [ ]:
import Base: +, *

+(Da::DiagonalMatrix, Db::DiagonalMatrix) = DiagonalMatrix(Da.diag + Db.diag)
*(x::Number, D::DiagonalMatrix) = DiagonalMatrix(x * D.diag)

In [ ]:
@which D + D

In [ ]:
@which 3 * D

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in precisely the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our `DiagonalMatrix` type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [ ]:
using BenchmarkTools

In [ ]:
operation(x) = x + 2*x

In [ ]:
x = rand(2,2)
@time operation.(x)

In [ ]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

In [ ]:
f()

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [ ]:
@benchmark operation.(x)

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [ ]:
@btime operation.(x);

However, we still have to take some care to avoid accessing global variables.

In [ ]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

This is similar to string interpolation:

In [ ]:
x = 42
s = "The answer to the ultimate question of life, the universe, and everything is $x !"

More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [ ]:
using LinearAlgebra
x = rand(100);
Djl = Diagonal(x)
D = DiagonalMatrix(x)
@btime operation($Djl);
@btime operation($D);

# Core messages of this Notebook

* **User defined types are as good as built-in types.**
* There are `mutable struct`s and immutable `struct`s.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* **Subtyping an existing interface** can give lots of functionality for free.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.